# Ensuring an interface without using virtual

Curiously_recurring_template_pattern 
https://en.wikipedia.org/wiki/Curiously_recurring_template_pattern

since C++0x
https://stackoverflow.com/questions/257288/templated-check-for-the-existence-of-a-class-member-function

since C++11
https://stackoverflow.com/questions/44317289/c-interface-without-virtual-functions

In [ ]:
#include <type_traits>

// workaround for C++0x
template <bool COND> class STATIC_ASSERT{ public: STATIC_ASSERT(){}; }; // Will compile for true
template <>          class STATIC_ASSERT<false>;                        // Won't compile for false due to missing definition


#define DECLARE_HAS_MEM_FUNC(func, name)                                        \
    template<typename T, typename Sign>                                         \
    struct name {                                                               \
        typedef char yes[1];                                                    \
        typedef char no [2];                                                    \
        template <typename U, U> struct type_check;                             \
        template <typename _1> static yes &chk(type_check<Sign, &_1::func > *); \
        template <typename   > static no  &chk(...);                            \
        static bool const value = sizeof(chk<T>(0)) == sizeof(yes);             \
    }

template<typename L>
class Interface
{
public:
  Interface() {
    typedef bool (L :: *foo_functPtr_t) (int);
#if (defined(_MSC_VER) && (_MSC_VER >= 1600)) || (__cplusplus>=201103L)
    static_assert(has_foo<L, foo_functPtr_t>::value, "member function foo not implemented");
#else
    STATIC_ASSERT<(has_foo<L, foo_functPtr_t>::value)>();
#endif
  };
private:
  DECLARE_HAS_MEM_FUNC(foo, has_foo);
};

class ImplementationOk : public Interface<ImplementationOk>
{
public:
  bool foo(int val) { return val > 0; };
};


class ImplementationFailed1 : public Interface<ImplementationFailed1>
{
public:
  bool foo(float val) { return val; };
};

class ImplementationFailed2 : public Interface<ImplementationFailed2>
{
};

int main()
{
  ImplementationOk a0;       // ok
  ImplementationFailed1 a1;  // fails due to wrong signature
  ImplementationFailed2 a2;  // fails due to missing foo function
}

